In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   
import warnings
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from catboost import CatBoostClassifier, Pool, cv
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', 100)
warnings.filterwarnings('ignore')

### Load data

In [2]:
path = 'https://raw.githubusercontent.com/McGill-MMA-EnterpriseAnalytics/customer_banking/uplifting_model/Dataset/bank-full.csv'
dataset = pd.read_csv(path,sep = ';')
dataset

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no


In [3]:
dataset['poutcome'].value_counts()

unknown    36959
failure     4901
other       1840
success     1511
Name: poutcome, dtype: int64

In [4]:
dataset['y'] = dataset['y'].map({'yes': 1, 'no': 0})
dataset

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,1
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,1
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,1
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,0


In [5]:
# preprocessing

# 30% of the data in 'contact' is unknown, so we will drop this column
dataset = dataset.drop(['contact'], axis=1)

# managing categorical variables
# convert binary variables to 0 and 1
dataset['default'] = dataset['default'].map({'yes': 1, 'no': 0}).astype(int)
dataset['housing'] = dataset['housing'].map({'yes': 1, 'no': 0}).astype(int)
dataset['loan'] = dataset['loan'].map({'yes': 1, 'no': 0}).astype(int)


### Causal Inference

Does a successful outcome in a previous marketing campaign increase the likelihood of a client subscribing to a term deposit in the current campaign?
*   Target: being subscription (yes or no)
*   Treatment group: having had a successful previous campaign outcome (success)
*   Control group: having had any other outcome (failure and other)

In [6]:
# convert 'poutcome' to a binary treatment variable and 'y' to binary
dataset['T'] = dataset['poutcome'].apply(lambda x: 1 if x == 'success' else 0)
dataset['y'] = dataset['y'].map({'yes': 1, 'no': 0})

In [7]:
VARS = [
    'age',
    'job',
    'marital',
    'education',
    'balance',
    'housing',
    'loan',
    #'contact',
    'day',
    'month',
    'duration',
    'campaign',
    'pdays',
    'previous'
    ]
CAT_VARS = [
    'job',
    'marital',
    'education',
    'housing',
    'loan',
    #'contact',
    'month'
    ]

In [8]:
# selecting variables and encode categorical ones
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), CAT_VARS)
    ],
    remainder='passthrough'
)


In [9]:
X = preprocessor.fit_transform(dataset[VARS])
X = pd.DataFrame(X, columns=preprocessor.get_feature_names_out())

# treatment and outcome
T = dataset['T']
y = dataset['y']

# I assume all variables except treatment can act as confounders for simplicity
W = X.copy()

### Approach 1: LinearDMLCateEstimator

In [15]:
import econml
print(econml.__version__)

0.15.0


In [11]:
from econml.dml import DML

In [14]:
from econml.dml import DMLCateEstimator
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor

# estimator
linear_dml_est = LinearDMLCateEstimator(
    model_y=GradientBoostingRegressor(),
    model_t=GradientBoostingRegressor(),
    featurizer=None  # since we already preprocessed our features
)


ImportError: cannot import name 'DMLCateEstimator' from 'econml.dml' (/Users/sheidamajidi/anaconda3/envs/causalml-py38/lib/python3.8/site-packages/econml/dml/__init__.py)

In [ ]:
# fitting the estimator
linear_dml_est.fit(y, T, X=None, W=W)

# estimating the treatment effects
linear_treatment_effects = linear_dml_est.effect(W)

# confidence intervals
linear_lb, linear_ub = linear_dml_est.effect_interval(W, alpha=0.05)


### Approach 2: ForestDMLCateEstimator

In [ ]:
from econml.dml import ForestDMLCateEstimator

# estimator
forest_dml_est = ForestDMLCateEstimator(
    model_y=GradientBoostingRegressor(),
    model_t=GradientBoostingRegressor(),
    featurizer=None 
)


In [ ]:
# fitting the estimator
forest_dml_est.fit(y, T, X=None, W=W, inference='blb')

# estimating the treatment effects
forest_treatment_effects = forest_dml_est.effect(W)

# confidence intervals
forest_lb, forest_ub = forest_dml_est.effect_interval(W, alpha=0.05)